# Determining Optimal Gate Bias 

In [3]:
import qcodes as qc
import numpy as np
from time import sleep

import sweep 
from sweep.sweep_load import pload1d
from barreralabdrivers.utils.param_utils import paramp
from cappy.config import dualhemtconfig, datapath
from cappy.balancer import Balancer

from qcodes.validators import Numbers
from qcodes.parameters import Parameter, ScaledParameter
import datetime

In [5]:
qc.Instrument.close_all()   # in case of existing connecitons
container = qc.Station(config_file=str(dualhemtconfig))

acdac = container.load_instrument("acdac")
dcdac = container.load_instrument("dcdac")
yoko = container.load_instrument("yoko")
keithley = container.load_instrument("keithley")
li1 = container.load_instrument("lockin")

print(100 * "-")
print(f"{datetime.datetime.now().strftime('%c')}")

Connected to: BARRERA ACDAC02 (serial:AD9106, firmware:2.00) in 3.08s
Connected to: BARRERA DCDAC01 (serial:AD5764, firmware:2.00) in 3.05s
Connected to: YOKOGAWA 765611 (serial:90ZC43353, firmware:1.10) in 0.05s
Connected to: KEITHLEY INSTRUMENTS DMM6500 (serial:04608216, firmware:1.7.12b) in 0.01s
Connected to: Stanford_Research_Systems SR865A (serial:005296, firmware:V1.51) in 0.10s
----------------------------------------------------------------------------------------------------
Fri Jan 24 18:18:18 2025


In [23]:
# Applied AC Voltages
Vref = acdac.ch1
V1pf = acdac.ch2
Vlref = acdac.ch3

# Applied DC Voltages
Vdd = yoko.channel1.voltage #drain to source voltage on amplifier hemt 
Vgamp = dcdac.channel1.voltage 

non_positive_validator = Numbers(min_value=-2, max_value=0)
vg_max_validator = Numbers(min_value = -2, max_value=Vgamp.get())

Vgamp.add_validator(non_positive_validator)  
Vgres = yoko.channel2.voltage
Vgres.add_validator(vg_max_validator)

# Measured Quantities 
Vdsdc = keithley.amplitude
Vdsac_R = li1.R
Vdsac_P = li1.P

In [24]:
def Vgamp_set(value):
    """
        Use this function to set Vgamp to value and update validator for Vgres. DO NOT set Vgamp natively. 
    """
    curr_Vgres = Vgres()
    if curr_Vgres > value:
        # update validator 
        vg_max_validator = Numbers(min_value = -3, max_value=value)
        Vgres.remove_validator()
        Vgres.add_validator(vg_max_validator)

        # set Vgamp and Vgres
        Vgres(value - 0.1)
        sleep(0.2)

    Vgamp(value)

In [25]:
Vgres.validators

(<Numbers -2<=v<=0.0>,)